# Very primitive particle-particle interaction in Peano 4

A very simple demonstration how to extend the simple particle setup with an explicit Euler and some simple particle-particle interaction. We use a fake potential, so there's no physical meaning in this example. However, it demonstrates the principles behind multibody simulations with the Peano toolbox.

In [1]:
import os

import peano4
import peano4.dastgen2
import peano4.toolbox
import peano4.toolbox.particles
import dastgen2


DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org


In [2]:
project = peano4.Project( ["examples", "particles"], "particle-particle-interaction", "." )

In [3]:
#build_mode = peano4.output.CompileMode.Asserts
build_mode = peano4.output.CompileMode.Release
project.output.makefile.parse_configure_script_outcome( "../../.." )
project.output.makefile.set_mode( build_mode )

parse configure outcome ../../../config.log to extract configure settings
found the configure call info   $ ./configure CXX=icpx --enable-exahype --enable-loadbalancing --with-multithreading=omp CXXFLAGS=--std=c++17 -fopenmp -DnoMPISupportsSingleSidedCommunication LDFLAGS=-fopenmp --enable-blockstructured --enable-particles

parse configure outcome ../../../src/Makefile to extract compile settings
add CXX=icpx
add CXXFLAGS=--std=c++17 -fopenmp -DnoMPISupportsSingleSidedCommunication
add CXXFLAGS_PEANO_2D_ASSERTS=-DDimensions=2 -DPeanoDebug=2 -g3 -O0
add CXXFLAGS_PEANO_2D_DEBUG=-DDimensions=2 -DPeanoDebug=4 -g3 -O0
add CXXFLAGS_PEANO_2D_RELEASE=-DDimensions=2 -DPeanoDebug=0
add CXXFLAGS_PEANO_2D_STATS=-DDimensions=2 -DPeanoDebug=0 -DTrackStatistics
add CXXFLAGS_PEANO_2D_TRACE=-DDimensions=2 -DPeanoDebug=1 -g3
add CXXFLAGS_PEANO_3D_ASSERTS=-DDimensions=3 -DPeanoDebug=2 -g3 -O0
add CXXFLAGS_PEANO_3D_DEBUG=-DDimensions=3 -DPeanoDebug=4 -g3 -O0
add CXXFLAGS_PEANO_3D_RELEASE=-DDimensions=3 -

## Model the particle

This time, we add an additional velocity field to the particle. We use Peano's double array here for the velocities to have full support of Peano's vector classes. Instead of saying that this array had two or three entries, we parameterise it through the symbol Dimensions. This way, our code will immediately run for 2d and 3d without any changes. Dimensions is a constant that Peano 4 defines. There are few of these, but Dimensions is really the important one.

In [4]:
particle  = peano4.toolbox.particles.Particle( "Particle" )
particle.data.add_attribute( peano4.dastgen2.Peano4DoubleArray("v","Dimensions") )
particles = peano4.toolbox.particles.ParticleSet( particle )

In [5]:
project.datamodel.add_global_object(particle)
project.datamodel.add_vertex(particles)

## Model the setup phases

The setup remains almost unchanged. The one tiny little difference is that we initialise the velocity properly when we create a particle. The generator above gives you different settings for v. We set the velocity component-wisely here.

In [6]:
mesh_size = 0.2

particle_tree_analysis = peano4.toolbox.particles.ParticleTreeAnalysis(particles)
project.datamodel.add_cell(particle_tree_analysis.cell_marker)   # read docu of ParticleTreeAnalysis

create_grid = peano4.solversteps.Step( name="CreateGrid", add_user_defined_actions=False )
create_grid.add_action_set( peano4.toolbox.CreateRegularGrid(mesh_size) )
create_grid.use_vertex(particles)
create_grid.use_cell(particle_tree_analysis.cell_marker)
project.solversteps.add_step(create_grid)

In [7]:
initialisation_snippet = """
 // we could do something intelligent here, but we actually
 // just set the cut-off radius
 particle->setCutOffRadius(0.001);
 particle->setV(0,((double) rand() / (RAND_MAX))-0.5);
 particle->setV(1,((double) rand() / (RAND_MAX))-0.5);
"""

init_setup = peano4.solversteps.Step( name="Init", add_user_defined_actions=False )
init_setup.use_vertex(particles)
init_setup.use_cell(particle_tree_analysis.cell_marker)
init_setup.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
init_setup.add_action_set( peano4.toolbox.particles.InsertRandomParticlesIntoUnrefinedCells(
  particle_set=particles,
  average_distance_between_particles=mesh_size/10,
  initialisation_call=initialisation_snippet,
  noise=True ))
init_setup.add_action_set( particle_tree_analysis )
project.solversteps.add_step(init_setup)

In [8]:
print_solution = peano4.solversteps.Step( "Plot", add_user_defined_actions=False )
print_solution.use_vertex(particles)
print_solution.use_cell(particle_tree_analysis.cell_marker)
print_solution.add_action_set( particle_tree_analysis )
print_solution.add_action_set( peano4.toolbox.PlotGridInPeanoBlockFormat( filename="grid", time_stamp_evaluation=peano4.toolbox.PlotGridInPeanoBlockFormat.CountTimeSteps ) )
print_solution.add_action_set( peano4.toolbox.particles.PlotParticlesInVTKFormat( "particles", particles, time_stamp_evaluation=peano4.toolbox.particles.PlotParticlesInVTKFormat.CountTimeSteps ) )
project.solversteps.add_step(print_solution)


## Particle behaviour

Our particle behaviour corresponds to a new algorithmic phase: the move. Compared to the other algorithmic steps, the big (technical) difference this time is that we ask Peano to give us a user-defined hook-in point for the action set. That is, not all the behaviour is realised via pre-manufactured action sets anymore.

In [9]:
move_particles = peano4.solversteps.Step( "Move" )
move_particles.use_vertex(particles)
move_particles.use_cell(particle_tree_analysis.cell_marker)
move_particles.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
move_particles.add_action_set( peano4.toolbox.particles.ParticleAMR(particles,particle_tree_analysis,min_particles_per_cell=10) )
move_particles.add_action_set( particle_tree_analysis )
project.solversteps.add_step(move_particles)

## Generate the actual C++ code

Next, we generate the actual Peano 4 C++ code:

In [10]:
project.generate()

generate all code ...
user has to modify class Move in actions directory manually 
generated particle-particle-interaction-main.cpp
write ./Makefile
./vertexdata/ParticleSet.h written by Jinja2TemplatedHeaderImplementationFilePair.py (from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.h)
./vertexdata/ParticleSet.cpp written by Jinja2TemplatedHeaderImplementationFilePair.py (from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.cpp)
./repositories/DataRepository.h written by TemplatedHeaderImplementationFilePair.py
./repositories/DataRepository.cpp written by TemplatedHeaderImplementationFilePair.py
././observers/CreateGrid2peano4_toolbox_CreateRegularGrid0.h written by ActionSet.py
././observers/CreateGrid2peano4_toolbox_CreateRegularGrid0.cpp written by ActionSet.py
write ././observers/CreateGrid.h
././observers/CreateGrid.cpp written by Observer.py
././observers/Init2peano4_toolbox_particles_UpdateParticleGrid

## Write the main

This time, the main is slightly more sophisticated. We continue to set up the grid as before, but then we always run 10 iterations of the Move, i.e. we do 10 time steps, and then we plot. The whole sequence is repeated a couple of times. Have a look into the main file in the repo to see the whole snippet:

    ...
    for (int i=0; i<100; i++) {
      for (int j=0; j<10; j++) {
        examples::particles::observers::Move moveObserver;
        peano4::parallel::SpacetreeSet::getInstance().traverse(moveObserver);
      }
      logInfo( "main()", "plot" )
      examples::particles::observers::Plot plotTimeStepObserver;
      peano4::parallel::SpacetreeSet::getInstance().traverse(plotTimeStepObserver);
    }


## Write the actual particle movement

We will implement the actual particle movement ourselves. For this, we have asked Move to be an action set with manual user code, and indeed we get a Move class in the actions subdirectory once we have triggered the build process. This is, same as main, an empty blueprint. We can alter it, and Peano won't overwrite it anymore after that. If we delete it, the API will regenerate it however. The current example has already a pre-prepared Move, so it won't be empty, but you might want to delete it and regenerate it to see what you get in the first place.

Here's what I changed in the Move class: Move is a standard class which defines an action set: There's a fixed number of callbacks that are invoked by Peano, and we can use these to inject behaviour. I only alter (or plug into) three actions: 

### First access to vertex

Particles are associated to vertices. So when I hit a vertex the first time, I also hit the particles associated with it for the first time. I run over them, and set their status to NotMovedYet:

     for (auto& p: fineGridVertexParticleSet) {
       p->setMoveState( globaldata::Particle::MoveState::NotMovedYet );
     }

### Hit cell

When I hit a cell, I actually evaluate the "forces" and alter the particles' velocities. I do this only for the particles that reside within the cell, as I know that a neighbouring cell will either be hit after or before the current one. If I modified all particles associated with the 2^d vertices of a cell, I'd modify particles multiple times. 

Please note that I only change the particles' attributes. I do not change their position. That means, the logical topology of all data, i.e. how they are assigned to vertices remains valid and intact. This means that we don't care how Peano runs through the cells. All data remains valid.

Please see touchCellFirstTime() for details.


### Last access to vertex

When I hit a vertex for the last time, I know that all 2^d adjacent cells have been "visited". Therefore, I can now finally update the vertex position. The vertex won't be used by neighbouring cells anymore and we are fine. Again, I do this only for particles that haven't moved yet. Otherwise we might move a particle, it ends up around another vertex that we haven't processed yet and then it is moved again.

    const double timeStepSize = 0.0001;
    for (auto& p: fineGridVertexParticleSet) {
      if (
        p->getMoveState()!=globaldata::Particle::MoveState::Moved
        and
        p->getParallelState()==globaldata::Particle::ParallelState::Local
      ) {
        p->setX( p->getX() + timeStepSize * p->getV() );
        for (int d=0; d<Dimensions; d++) {
          if ( p->getX()(d)<0.0 ) {
            p->setV(d, std::abs(p->getV()(d)));
          }
          if ( p->getX()(d)>1.0 ) {
            p->setV(d, -std::abs(p->getV()(d)));
          }
        }
      }
      p->setMoveState( globaldata::Particle::MoveState::Moved );
    }

You see, we also add reflecting boundary conditions.


## Build the code

In [11]:
project.build()

clean up project ...
rm -f celldata/ParticleSetCellStatistics.o vertexdata/ParticleSet.o globaldata/Particle.o repositories/DataRepository.o ./observers/CreateGrid2peano4_toolbox_CreateRegularGrid0.o ./observers/CreateGrid.o ./observers/Init2peano4_toolbox_particles_UpdateParticleGridAssociation0.o ./observers/Init2peano4_toolbox_particles_InsertRandomParticlesIntoUnrefinedCells1.o ./observers/Init2peano4_toolbox_particles_ParticleAMR2.o ./observers/Init2peano4_toolbox_particles_ParticleTreeAnalysis3.o ./observers/Init.o ./observers/Plot2peano4_toolbox_PlotGridInPeanoBlockFormat0.o ./observers/Plot2peano4_toolbox_particles_PlotParticlesInVTKFormat1.o ./observers/Plot.o ./actions/Move.o ./observers/Move2peano4_toolbox_particles_UpdateParticleGridAssociation1.o ./observers/Move2peano4_toolbox_particles_ParticleAMR2.o ./observers/Move2peano4_toolbox_particles_ParticleTreeAnalysis3.o ./observers/Move.o repositories/StepRepository.o particle-particle-interaction-main.o    
rm -f *.mod
rm -f

## Run code

In [ ]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.endswith(".pvd") ]
for f in output_files:
  os.remove(f)


In [ ]:
# success = project.run( args=["--threads", "1"], prefix=["mpirun", "-n", "1"] )
!./peano4

# Wrap-up

Our introductions here follow a standard Peano development pattern. Very often, we write codes via the right action sets. Once these have converged and are mature, we refactor them out into the Python API as toolset. That's how the whole particle toolset emerged. This way, they can be used easily by other users. Consequently, our next example on multiscale particles won't use a hand-written action set anymore. It will be use a pre-manufactured action set from the toolset and insert code snippets into this one.

If you study the code en detail (and let it run for a while), you'll recognise that something is wrong. This is where we need multiscale data ...